In [1]:
# Remove verbose logging of tensorflow and absl
# (Greatly helpful on preventing freezing ipykernel while kernel sampling process)
import warnings
from silence_tensorflow import silence_tensorflow
warnings.filterwarnings('ignore')
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)
silence_tensorflow()

In [2]:
# initialize builder config with workspace
from nn_meter.builder import builder_config
import os
builder_config.init(os.getcwd()) 

In [3]:
# build latency predictor for kernel
from nn_meter.builder.utils import merge_prev_info
from nn_meter.builder.nn_meter_builder import sample_and_profile_kernel_data
from nn_meter.builder.kernel_predictor_builder import build_predictor_by_data
import logging

backend = "ssh_tflite_cpu"
all_kernel_types = ["conv_bn_relu", "dwconv_bn_relu", "fc_block", "globalavgpool_block", "hswish_block", "relu_block",
                    "se_block", "split_block", "add_block", "add_relu", "maxpool_block", "avgpool_block",
                    "bn_block", "bn_relu", "channel_shuffle", "concat_block"]

for kernel_type in all_kernel_types:
    print(f"Currently running {kernel_type} ...")
    
    """ 
    Build latency predictor for given kernel. This method contains three main steps:
    1. sample kernel configs and profile kernel model based on configs;
    2. initialize latency predictor of kernel based on the profiled data;
    3. adopt adaptive sampler with iteratively doing step 1 for finegrained sampling to improve predictor performance

    @params
    
    kernel_type (str): the type of kernel
    
    backend (str): the name of backend instance to profile models
    
    init_sample_num (int, optional): the data size for predictor initialization. Defaults to 1000.
    
    finegrained_sample_num (int, optional): the data size for adaptive sampling. For each data with error higher than 
        error_threshold, #finegrained_sample_num data will be generated based the the large error data. Defaults to 10.

    iteration (int, optional): the iteration for adaptive sampler. Defaults to 5.

    error_threshold (float, optional): the threshold of large error. Defaults to 0.2.
    """
    init_sample_num = 10 # 1000
    finegrained_sample_num = 10
    iteration = 5
    error_threshold = 0.1
    
    ws_mode_path = builder_config.get('MODEL_DIR', 'predbuild')
    

    # init predictor builder with prior data sampler
    kernel_data = sample_and_profile_kernel_data(kernel_type, init_sample_num, backend, sampling_mode='prior', mark='prior')

    # use current sampled data to build regression model, and locate data with large errors in testset
    predictor, acc10, error_configs = build_predictor_by_data(kernel_type, kernel_data, backend, error_threshold=error_threshold, mark='prior',
                                                              save_path=os.path.join(ws_mode_path, "results"))
    logging.keyinfo(f'Iteration 0: acc10 {acc10}, error_configs number: {len(error_configs)}')

    for i in range(1, iteration):
        # finegrained sampling and profiling for large error data
        new_kernel_data = sample_and_profile_kernel_data(kernel_type, finegrained_sample_num, backend,
                                                  sampling_mode = 'finegrained', configs=error_configs, mark=f'finegrained{i}')

        # merge finegrained data with previous data and build new regression model
        kernel_data = merge_prev_info(new_info=new_kernel_data, prev_info=kernel_data)
        predictor, acc10, error_configs = build_predictor_by_data(kernel_type, kernel_data, backend, error_threshold=error_threshold, mark=f'finegrained{i}',
                                                                  save_path=os.path.join(ws_mode_path, "results"))
        logging.keyinfo(f'Iteration {i}: acc10 {acc10}, error_configs number: {len(error_configs)}')

    # return predictor, kernel_data


Currently running fc_block ...
(nn-Meter) building kernel for fc_block...
(nn-Meter) fc_block model is generated and saved to /home/jungin500/Workspace/nni-workspace/nn-meter/workspaces/rpi3b-workspace-v2/predictor_build/models/fc_block_prior_DZ837F.
(nn-Meter) fc_block model is generated and saved to /home/jungin500/Workspace/nni-workspace/nn-meter/workspaces/rpi3b-workspace-v2/predictor_build/models/fc_block_prior_MID9EO.
(nn-Meter) fc_block model is generated and saved to /home/jungin500/Workspace/nni-workspace/nn-meter/workspaces/rpi3b-workspace-v2/predictor_build/models/fc_block_prior_T8JAI6.
(nn-Meter) fc_block model is generated and saved to /home/jungin500/Workspace/nni-workspace/nn-meter/workspaces/rpi3b-workspace-v2/predictor_build/models/fc_block_prior_D8B3T0.
(nn-Meter) fc_block model is generated and saved to /home/jungin500/Workspace/nni-workspace/nn-meter/workspaces/rpi3b-workspace-v2/predictor_build/models/fc_block_prior_V48Z6R.
(nn-Meter) fc_block model is generated an